<table style="width: 100%;"> <tr> <td style="width: 20%; vertical-align: top;"> <img src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" alt="UdeA" height="150px"> </td> <td style="width: 80%; padding-left: 20px;"> <strong style="font-size: 30px;">DESPLIGUE PARA UN SISTEMA DE CLASIFICACIÓN CON IMÁGENES DE RESONANCIA MAGNÉTICA </strong><br><br> <span style="font-size: 30px;"> Luisa Fernanda Alzate Cuartas <br> Juan Camilo Henao Caro<br>Isabella Mendez Hoyos<br> Fernando Antonio Piñeres Ramos </span> </td> </tr> </table>

#### LIBRERÍAS REQUERIDAS

In [5]:
#Conectar a Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#Librería requeridas.
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import openpyxl

#### DESPLIGUE DEL MODELO

In [ ]:
#Definición de rutas
base_path = '/content/drive/MyDrive/operaciones/salidas/data_final' #Ruta del conjunto de datos.

ruta_x = '/content/drive/MyDrive/operaciones/salidas/data_final/x_val.pkl'
ruta_y = '/content/drive/MyDrive/operaciones/salidas/data_final/y_val.pkl'
ruta_modelo =
ruta_resultado = base_path + 'despliegue/clasificados_val.xlsx'

In [ ]:
if __name__ == "__main__":

    # Cargar imágenes para despliegue
    path = 'data/despliegue/'
    x, _, files = fn.img2data(path)  # carga imágenes
    x = np.array(x).astype('float32') / 255.0  # escala a [0,1]